# 文档加载 Excel CSV PDF 等

In [22]:
from langchain_community.document_loaders import TextLoader

from src.llm.llm_factory import LLMFactory

TextLoader("D:\\AiWork\\LanchainBuild\\README.md").load()

[Document(metadata={'source': 'D:\\AiWork\\LanchainBuild\\README.md'}, page_content='# LanchainBuild')]

In [ ]:
# 加载 CSV 文件
from langchain.document_loaders import CSVLoader

CSVLoader("").load()

In [ ]:
# 需要安装相关包
# pip install docx2txt
# pip install pypdf
# pip install nltk

In [21]:
# 加载excel 表格
# 安装包 pip install "unstructured[xlsx]"
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import DirectoryLoader
docs = DirectoryLoader("D:\\OneDrive - Lenovo\\桌面\\icon\\项目文档\\langchain_load\\", glob="*.xlsx", loader_cls=UnstructuredExcelLoader).load()


len(docs)

Error loading file D:\OneDrive - Lenovo\桌面\icon\项目文档\langchain_load\2021-Q1-DCG.xlsx


BadZipFile: File is not a zip file

In [1]:
# 加载html 文件
from langchain_community.document_loaders import UnstructuredHTMLLoader

docs = UnstructuredHTMLLoader("D:\\AiWork\\edw_agent\\src\\server\\static\\index.html").load()
print(docs)

BadZipFile: File is not a zip file

In [ ]:
# 加载pdf文件
# pip install pypdf
from langchain_community.document_loaders import PyPDFLoader



In [2]:
# 切割文档

from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('D:\\AiWork\\LanchainBuild\\README.md', 'r', encoding='utf-8') as f:
    text = f.read()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 50, # 切分的文本块大小， 一般通过长度函数计算
        chunk_overlap = 20, # 重叠的文本块大小
        length_function = len, # 文本长度函数
        add_start_index = True, # 是否添加开始索引
    )
    # 切割完毕
    result = text_splitter.create_documents(text)

In [ ]:
# 按字符切割
from langchain.text_splitter import CharacterTextSplitter
with open('D:\\AiWork\\LanchainBuild\\README.md', 'r', encoding='utf-8') as f:
    text = f.read()
    text_splitter = CharacterTextSplitter(
        separator = ",", #切割的标识符
        chunk_size = 50, # 切分的文本块大小， 一般通过长度函数计算
        chunk_overlap = 20, # 重叠的文本块大小
        length_function = len, # 文本长度函数
        add_start_index = True, # 是否添加开始索引
        is_separator_regex = False, # 是否使用正则表达式作为分隔符
    )
    # 切割完毕
    result = text_splitter.create_documents(text)

In [3]:
# 代码文档的切割

from langchain.text_splitter import (
    Language,
    RecursiveCharacterTextSplitter,
)
code = """
def hello_world():
    print("hello world")
# 调用函数
hello_world()
"""
py_splitter = RecursiveCharacterTextSplitter.from_language(
    language = Language.PYTHON,
    chunk_size = 50,
    chunk_overlap = 10,
)

result = py_splitter.create_documents([code])
result

[Document(metadata={}, page_content='def hello_world():\n    print("hello world")'),
 Document(metadata={}, page_content='# 调用函数\nhello_world()')]

In [4]:
# 按token进行切割
from langchain.text_splitter import CharacterTextSplitter

with open('D:\\AiWork\\LanchainBuild\\README.md', 'r', encoding='utf-8') as f:
    text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size = 4000, # 切分的文本块大小， 一般通过长度函数计算
        chunk_overlap = 30, # 重叠的文本块大小
    )
    # 切割完毕
    result = text_splitter.create_documents(text)

In [ ]:
# 实战 要用  doctran

with open("") as f:
    content = f.read()
    

In [9]:
# 长下下文，切割精度丢失问题
# 需要安装 pip install sentence-transformers

# from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.document_transformers import (
    long_context_reorder
)
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

#使用hugging face托管的LLm 来做嵌入
embedding = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
    
text = [
"篮球是一项伟大的运动。",
                      "带我飞往月球是我最喜欢的歌曲之一。",
                      "凯尔特人队是我最喜欢的球队。",
                      "这是一篇关于波士顿凯尔特人的文件。",
                      "我非常喜欢去看电影。",
                      "波士顿凯尔特人队以20分的优势赢得了比赛。",
                      "这只是一段随机的文字。",
                      "《艾尔登之环》是过去15年最好的游戏之一。",
                     "L.科内特是凯尔特人队最好的球员之一。",
                     "拉里伯德是一位标志性的NBA球员。"
]

retriever = Chroma.from_texts(text, embedding = embedding).as_retriever(
    search_kwargs={"k": 3}  # 设置检索的数量
)
query = "关于凯尔特人队你知道什么?"
docs = retriever.get_relevant_documents(query)
# 不好搞下载不了


No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Check your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
# 对检索结果进行排序
# 问题相关性的越低的，放中间，相关性越高的放两边
recorder = long_context_reorder.LongContextReorder()
reo_docs = recorder.transform_documents(docs)

#头尾有四个
reo_docs

In [13]:
from src.llm.llm_factory import LLMFactory
model = LLMFactory.create_llm()

In [14]:
# 调用大模型校验一下精度
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, StuffDocumentsChain
document_prompt = PromptTemplate(input_variables=["page_content"], template="{page_content}")

stuff_prompt_override = """Give this text extracts:
---------------------------
{context}
---------------------------
Please answer the following questions:
{query}
"""
prompt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["context", "query"],
)
llm_chain = LLMChain(
    llm = model,
    prompt = prompt,
)
work_chain = StuffDocumentsChain(
    llm_chain = llm_chain,
    document_prompt = document_prompt,
    document_variable_name = "context",
)
work_chain.run(
    input_documents = reo_docs,
    query = "凯尔特人队是哪里的球队？"
)


C:\Users\longyu3\AppData\Local\Temp\ipykernel_17812\51598467.py:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
C:\Users\longyu3\AppData\Local\Temp\ipykernel_17812\51598467.py:21: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  work_chain = StuffDocumentsChain(


NameError: name 'reo_docs' is not defined